쿼리 검증<br>
SQL 체인이나 에이전트에서 가장 오류가 발생하기 쉬운 부분은 유효하고 안전한 SQL 쿼리를 작성하는 것입니다. 이 가이드에서는 쿼리를 검증하고 잘못된 쿼리를 처리하는 몇 가지 전략에 대해 설명합니다.

In [ ]:
import os
from langchain_community.utilities import SQLDatabase

# OpenAI API 키 설정 (이 예제에서는 사용되지 않음)
api_key = os.getenv("OPENAI_API_KEY")

# SQLite 데이터베이스 경로
db_path = "/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/Chinook.db"

# SQLite 데이터베이스 연결
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# 데이터베이스 방언 출력 (예: sqlite)
print(db.dialect)

# 사용 가능한 테이블 이름 출력
print(db.get_usable_table_names())

# SQL 쿼리 실행 및 결과 출력
result = db.run("SELECT * FROM Artist LIMIT 10;")
print(result)

쿼리 검사기<bR>
가장 간단한 전략 중 하나는 모델 자체에 원본 쿼리의 일반적인 실수를 확인하도록 요청하는 것입니다. 예를 들어, 다음과 같은 SQL 쿼리 체인이 있다고 가정해 봅시다:

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = create_sql_query_chain(llm, db)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)
validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [ ]:
query = full_chain.invoke(
    {
        "question": "What's the average Invoice from an American customer whose Fax is missing since 2003 but before 2010"
    }
)
query



In [ ]:
db.run(query)

대규모 데이터베이스<br>
데이터베이스에 대해 유효한 쿼리를 작성하려면 모델에 테이블 이름, 테이블 스키마 및 쿼리할 피처 값을 제공해야 합니다. 많은 테이블, 컬럼 또는 고유도가 높은 컬럼이 있을 때, 데이터베이스에 대한 모든 정보를 매번 프롬프트에 덤프하는 것은 불가능합니다. 대신, 프롬프트에 가장 관련 있는 정보만 동적으로 삽입하는 방법을 찾아야 합니다. 이를 위한 몇 가지 기술을 살펴보겠습니다.

In [19]:
from langchain_community.utilities import SQLDatabase

# db_path = "/workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/Chinook.db"

# # SQLite 데이터베이스 연결
# db = SQLDatabase.from_uri(f"sqlite:///{db_path}")


db = SQLDatabase.from_uri("sqlite:////workspace/youngwoo/toyproject-datallm/modules/agent/tmp_dataset/Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

많은 테이블<br>
프롬프트에 포함해야 하는 주요 정보 중 하나는 관련 테이블의 스키마입니다. 테이블이 매우 많을 때, 모든 스키마를 단일 프롬프트에 포함할 수 없습니다. 이러한 경우에는 먼저 사용자 입력과 관련된 테이블 이름을 추출하고, 그런 다음 해당 테이블의 스키마만 포함합니다.<Br>

이를 수행하는 간단하고 신뢰할 수 있는 방법 중 하나는 OpenAI 함수 호출 및 Pydantic 모델을 사용하는 것입니다. LangChain에는 이를 수행할 수 있는 내장 create_extraction_chain_pydantic 체인이 있습니다:

In [20]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")


table_names = "\n".join(db.get_usable_table_names())
system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""
table_chain = create_extraction_chain_pydantic(Table, llm, system_message=system)
table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[Table(name='Genre'), Table(name='Artist'), Table(name='Track')]

이 방법은 꽤 잘 작동합니다! 하지만 아래에서 볼 수 있듯이, 실제로는 몇 가지 다른 테이블도 필요합니다. 이는 모델이 사용자 질문만으로 알기 어려울 수 있습니다. 이 경우, 모델의 작업을 단순화하기 위해 테이블을 그룹으로 묶는 것을 고려할 수 있습니다. 모델에게 "Music"과 "Business" 카테고리 중에서 선택하도록 요청한 다음, 관련 테이블을 선택하도록 할 수 있습니다:

In [21]:
system = """Return the names of the SQL tables that are relevant to the user question. \
The tables are:

Music
Business"""
category_chain = create_extraction_chain_pydantic(Table, llm, system_message=system)
category_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[Table(name='Music'), Table(name='Business')]

In [22]:
from typing import List


def get_tables(categories: List[Table]) -> List[str]:
    tables = []
    for category in categories:
        if category.name == "Music":
            tables.extend(
                [
                    "Album",
                    "Artist",
                    "Genre",
                    "MediaType",
                    "Playlist",
                    "PlaylistTrack",
                    "Track",
                ]
            )
        elif category.name == "Business":
            tables.extend(["Customer", "Employee", "Invoice", "InvoiceLine"])
    return tables


table_chain = category_chain | get_tables  # noqa
table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

['Album',
 'Artist',
 'Genre',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track',
 'Customer',
 'Employee',
 'Invoice',
 'InvoiceLine']

이제 모든 쿼리에 대해 관련 테이블을 출력할 수 있는 체인이 생겼으므로, create_sql_query_chain과 결합할 수 있습니다. 이는 table_names_to_use 목록을 받아 프롬프트에 포함할 테이블 스키마를 결정할 수 있습니다:



In [23]:
from operator import itemgetter

from langchain.chains import create_sql_query_chain
from langchain_core.runnables import RunnablePassthrough

query_chain = create_sql_query_chain(llm, db)
# Convert "question" key to the "input" key expected by current table_chain.
table_chain = {"input": itemgetter("question")} | table_chain
# Set table_names_to_use using table_chain.
full_chain = RunnablePassthrough.assign(table_names_to_use=table_chain) | query_chain

In [24]:
# Alanis Morisette 노래의 모든 장르는 무엇인가요?
query = full_chain.invoke(
    {"question": "What are all the genres of Alanis Morisette songs"}
)
print(query)

SELECT g.Name
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
WHERE ar.Name = 'Alanis Morissette'


In [25]:
db.run(query)

"[('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',), ('Rock',)]"

질문을 약간 바꾸어 답변의 중복을 제거할 수 있습니다:

In [26]:
# Alanis Morisette 노래의 모든 고유한 장르의 집합은 무엇인가요?
query = full_chain.invoke(
    {"question": "What is the set of all unique genres of Alanis Morisette songs"}
)
print(query)

SELECT DISTINCT g.Name
FROM Genre g
JOIN Track t ON g.GenreId = t.GenreId
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Artist ar ON a.ArtistId = ar.ArtistId
WHERE ar.Name = 'Alanis Morissette'


In [27]:
db.run(query)

"[('Rock',)]"

고유도가 높은 컬럼<BR>
주소, 노래 이름 또는 아티스트와 같은 고유 명사를 포함하는 컬럼을 필터링하려면, 먼저 철자를 확인하여 데이터를 정확하게 필터링해야 합니다.<bR>

하나의 단순한 전략은 데이터베이스에 존재하는 모든 고유 명사로 벡터 스토어를 생성하는 것입니다. 그런 다음 사용자의 입력을 쿼리하여 가장 관련 있는 고유 명사를 프롬프트에 삽입할 수 있습니다.<BR>

먼저 원하는 엔티티의 고유 값을 가져오는 함수를 정의합니다:

In [28]:
import ast
import re

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return res

In [29]:
proper_nouns = query_as_list(db, "SELECT Name FROM Artist")
proper_nouns += query_as_list(db, "SELECT Title FROM Album")
proper_nouns += query_as_list(db, "SELECT Name FROM Genre")
len(proper_nouns)
proper_nouns[:5]

['AC/DC', 'Accept', 'Aerosmith', 'Alanis Morissette', 'Alice In Chains']

['AC/DC', 'Accept', 'Aerosmith', 'Alanis Morissette', 'Alice In Chains']

이제 모든 값을 벡터 데이터베이스에 임베딩하고 저장할 수 있습니다:


In [31]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vector_db = FAISS.from_texts(proper_nouns, OpenAIEmbeddings())
retriever = vector_db.as_retriever(search_kwargs={"k": 15})


그리고 데이터베이스에서 값을 먼저 검색하고 이를 프롬프트에 삽입하는 쿼리 구성 체인을 구성합니다:

In [32]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

system = """You are a SQLite expert. Given an input question, create a syntactically \
correct SQLite query to run. Unless otherwise specificed, do not return more than \
{top_k} rows.\n\nHere is the relevant table info: {table_info}\n\nHere is a non-exhaustive \
list of possible feature values. If filtering on a feature value make sure to check its spelling \
against this list first:\n\n{proper_nouns}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

query_chain = create_sql_query_chain(llm, db, prompt=prompt)
retriever_chain = (
    itemgetter("question")
    | retriever
    | (lambda docs: "\n".join(doc.page_content for doc in docs))
)
chain = RunnablePassthrough.assign(proper_nouns=retriever_chain) | query_chain


체인을 시도해보려면, "elenis moriset"이라는 철자가 틀린 Alanis Morissette로 필터링할 때와 검색을 사용한 경우를 확인해 보겠습니다:

In [36]:
# Without retrieval
query = query_chain.invoke(
    {"question": "What are all the genres of elenis moriset songs", "proper_nouns": ""}
)
print(query)

# 불필요한 ```sql 제거
query = query.replace("```sql", "").replace("```", "")

# 쿼리 실행
result = db.run(query)
print(result)


```sql
SELECT DISTINCT g.Name AS Genre
FROM Track t
JOIN Album a ON t.AlbumId = a.AlbumId
JOIN Genre g ON t.GenreId = g.GenreId
WHERE t.Composer LIKE '%Elenis Moriset%'
```



In [38]:
# With retrieval
query = chain.invoke({"question": "What are all the genres of elenis moriset songs"})
# 불필요한 ```sql 제거
query = query.replace("```sql", "").replace("```", "")

# 쿼리 실행
result = db.run(query)
print(result)


[('All I Really Want', 'Rock'), ('You Oughta Know', 'Rock'), ('Perfect', 'Rock'), ('Hand In My Pocket', 'Rock'), ('Right Through You', 'Rock'), ('Forgiven', 'Rock'), ('You Learn', 'Rock'), ('Head Over Feet', 'Rock'), ('Mary Jane', 'Rock'), ('Ironic', 'Rock'), ('Not The Doctor', 'Rock'), ('Wake Up', 'Rock'), ('You Oughta Know (Alternate)', 'Rock')]
